<h1><center> PPOL 5203 Data Science I: Foundations <br><br> 
<font color='grey'> Collecting Digital Data - API<br><br>
Tiago Ventura </center> <h1> 

---

## Learning Goals

In the class today, we will learn how to collect digital data through APIs. We will focus on: 

- Building a solid understanding about APIs
- Working with three types of APIs:
    - APIs with no credentials and no wrappers
    - APIs with credentials and no wrappers
    - APIs with wrappers. 

In [1]:
# setup
import requests
import os
import pandas as pd

## APIs 101

The famous acronym API stands for “Application Programming Interface”. An API is an online server allows different applications to interact. Most often for our purposes, an API will facilitate information exchange between data users and the holders of certain data. Many companies build these repositories for various functions, including sharing data, receiving data, joint database management, and providing artificial intelligence functions or machines for public use.

Let's think of an example capable of motivating the creation of an API. Imagine you own Twitter. You would have zillions of hackers every day trying to scrape your data, this would make your website more unstable and insecure. What is a possible solution? You create an API, and you control who accesses the information, when they access it, and what type of information you make available. Another option is to close you API and restrict data access to researchers. But, if you do this, you are likely to pay a reputational cost for not being transparent, and users might leave your platform.

Have you ever watched Matrix? APIs are just like that! In the movies, Neil and others would physically connect their mindes to a super developed server  and ask to learn a certain skill - kung-fu, programming, language, etc. This is exactly what an API does. You connect to the website and request data, and receive it in return. It's like sending an email, but doing everything via programming language.

### API Use-Cases

There are two main ways in which we academics commonly use APIs.

1. Access data shared by Companies and NGOs.

2. Process our data in Algorithms developed by third parties.

Our focus will be on the first. Later, we will see how to use the ChatGPT API for text classification tasks. 


### APIs Components

An API is just an URL. See the example below:

`http://mywebsite.com/endpoint?key&param_1&param_2`

Main Components: 

- **http://mywebsite/**: API root. The domain of your api/
- **endpoint**: An endpoint is a server route for retrieving specific data from an API
- **key**: credentials that some websites ask for you to create before you can query the api. 
- **?param_1*param_2** parameters. Those are filters that you can input in apis requests. 


### Requests to APIs

In order to work with APIs, we need tools to access the web. In Python, the most common library for making requests and working with APIs is the `requests` library. There are two main types of requests: 

- `get()`: to receive information from the API -- which we will use the most for web data collection

- `post()`: to send information to the API -- think about the use of ChatGPT for classification of text. 


## Example 1: Open Trivia API

### Querying an API: Step-by-Step

Let's start querying our first API. We will start with the simple [Open Trivia API](https://opentdb.com/api_config.php). This is a very simple API, and serves the purpose of learning all the basic steps of querying APIs. The Open Trivia API gives you ideas for your trivia games!

The Trivia API: 

- **Does not** require us to create credentials.
- And **does not** have a Python wrapper.

When querying an API, our work will often involve the following steps: 

- **Step 1:** Look at the API documentation and endpoints, and construct a query of interest
- **Step 2:** Use requests.get(querystring) to call the API
- **Step 3:** Examine the response
- **Step 4:** Extract your data and save it. 

### Step 1: Documentation, Endpoints and Query. 

Before we start querying an API, we always need to read through the [documentation/reference](https://opentdb.com/api_config.php). The documentation often revel to us: 

- The base url for the API: `https://opentdb.com/api.php`
- The different endpoints: 
    - This api has only one endpoint
- The API parameters:
    - `amount`
    - `category`
    - And some others we will learn.
    
Notice one thing here. The Trivia API requires you to gave the `amount` filter in your call. Not all APIs are like this. Some have a random api endpoint for you to play around

In [2]:
# build query
query = "https://opentdb.com/api.php?amount=1"

### **Step 2:** Use `requests.get(querystring)` to call the API

To interact with the API, we will use the `requests` package. The requests package allow us to send a HTTP request to the API. Because we are intereste in retrieving data, we will mostly be working with the `.get()` method, which requires one argument — the URL we want to make the request to. 

When we make a request, the response from the API comes with a response code which tells us whether our request was successful. Response codes are important because they immediately tell us if something went wrong.

To make a ‘GET’ request, we’ll use the requests.get() function, which requires one argument — the URL we want to make the request to. We’ll start by making a request to an API endpoint that doesn’t exist, so we can see what that response code looks like

In [4]:
# Make a get request to get the latest position of the ISS from the OpenNotify API.
response = requests.get(query)
type(response)

requests.models.Response

### **Step 3:** Examine the response

When we make a request, the response from the API comes with a response code which tells us whether our request was successful. Response codes are important because they immediately tell us if something went wrong. Here is a list of response codes you can get

200 — Everything went okay, and the server returned a result (if any).

301 — The server is redirecting you to a different endpoint. This can happen when a company switches domain names, or when an endpoint's name has changed.

401 — The server thinks you're not authenticated. This happens when you don't send the right credentials to access an API.

400 — The server thinks you made a bad request. This can happen when you don't send the information that the API requires to process your request (among other things).

403 — The resource you're trying to access is forbidden, and you don't have the right permissions to see it.

404 — The server didn't find the resource you tried to access.


In [5]:
# check status code
status_code = response.status_code

# print status code
status_code

200

### **Step 4:** Extract your data.

With an 200 code, we can access the content of the get request. The return from the API is stored as a `content` attribute in the response object.

In [6]:
print(response.content)

b'{"response_code":0,"results":[{"type":"multiple","difficulty":"hard","category":"History","question":"Who invented the &quot;Flying Shuttle&quot; in 1738; one of the key developments in the industrialization of weaving?","correct_answer":"John Kay","incorrect_answers":["James Hargreaves","Richard Arkwright","John Deere"]}]}'


#### Processing JSONs

The deafault data type we receive from APIS are in the JSON format. This format encodes data structures like lists and dictionaries as strings to ensure that machines can read them easily. 

For that kind of content, the requests library includes a specific .json() method that you can use to immediately convert the API bytes response into a Python data structure, in general a nested dictionary. 

In [7]:
# convert the get output to a dictionary
response_dict = response.json()
print(response_dict)

{'response_code': 0, 'results': [{'type': 'multiple', 'difficulty': 'hard', 'category': 'History', 'question': 'Who invented the &quot;Flying Shuttle&quot; in 1738; one of the key developments in the industrialization of weaving?', 'correct_answer': 'John Kay', 'incorrect_answers': ['James Hargreaves', 'Richard Arkwright', 'John Deere']}]}


In [8]:
# index just like a dict
response_dict["results"][0]["question"]

'Who invented the &quot;Flying Shuttle&quot; in 1738; one of the key developments in the industrialization of weaving?'

In [9]:
# convert to a dataframe
import pandas as pd

# need to convert to a list for weird python reasons
pd.DataFrame([response_dict["results"][0]])

,type,difficulty,category,question,correct_answer,incorrect_answers
0,multiple,hard,History,Who invented the &quot;Flying Shuttle&quot; in...,John Kay,"[James Hargreaves, Richard Arkwright, John Deere]"


Let's see the full code:

In [10]:
# full code
import requests
import pandas as pd

# build query
query = "https://opentdb.com/api.php?amount=1"

# 
response = requests.get(query)

# check status code
status_code = response.status_code

# move forward with code
if status_code==200:
    # convert the get output to a dictionary
    response_dict = response.json()
    # convert to a dataframe
    res = pd.DataFrame([response_dict["results"][0]])
else:
    print(status_code)
    
# print the activity
res

,type,difficulty,category,question,correct_answer,incorrect_answers
0,multiple,easy,Entertainment: Music,What was Daft Punk&#039;s first studio album?,Homework,"[Discovery, Random Access Memories, Human Afte..."


### Exploring API Filters

If we look at the documentation, you see the APIs provides filters (query parameters) that allow you to refine your search. 

For example, when you send a `get` request to the Youtube API, you are not interested in the entire Youtube data. You want data associated with certain videos, profiles, for a certain period of time, for example. These filters are often embedded as query parameters in the API call. 

To add a query parameter to a given URL, you have to add a question mark (?) before the first query parameter. If you want to have multiple query parameters in your request, then you can split them with an ampersand (&)

We can add filters by: 

- constructing the full API call

- Using dictionaries


### Filter with the full API cal

In [11]:
## get only recreational activities
# build query
query = "https://opentdb.com/api.php"

# add filter
activity = "?amount=10"

# full request
url = query + activity

# Make a get request 
response = requests.get(url)

# see json
response.json()

{'response_code': 0,
 'results': [{'type': 'boolean',
   'difficulty': 'medium',
   'category': 'Entertainment: Video Games',
   'question': 'The game series &quot;Titanfall&quot; runs on the Unreal Engine.',
   'correct_answer': 'False',
   'incorrect_answers': ['True']},
  {'type': 'multiple',
   'difficulty': 'hard',
   'category': 'Science &amp; Nature',
   'question': 'What is the most potent toxin known?',
   'correct_answer': 'Botulinum toxin',
   'incorrect_answers': ['Ricin', 'Cyanide', 'Asbestos']},
  {'type': 'multiple',
   'difficulty': 'hard',
   'category': 'Entertainment: Video Games',
   'question': '&quot;Banjo-Kazooie&quot; originally started its development as a totally different game for the SNES under which codename?',
   'correct_answer': 'Project Dream',
   'incorrect_answers': ['Project Ukulele', 'Project Land', 'Project BK']},
  {'type': 'multiple',
   'difficulty': 'easy',
   'category': 'Entertainment: Film',
   'question': 'Which James Bond film had the them

### Or using dictionaries

In [12]:
## get only recreational activities
# build query
query = "https://opentdb.com/api.php"

# add filter
parameters = {"amount": "10", 
             "category":"9"}

# Make a get request to get 
response = requests.get(query, params=parameters)

# see json
print(response.status_code)
response.json()

200


{'response_code': 0,
 'results': [{'type': 'boolean',
   'difficulty': 'easy',
   'category': 'General Knowledge',
   'question': 'It is automatically considered entrapment in the United States if the police sell you illegal substances without revealing themselves.',
   'correct_answer': 'False',
   'incorrect_answers': ['True']},
  {'type': 'multiple',
   'difficulty': 'easy',
   'category': 'General Knowledge',
   'question': 'In which cardinal direction does the Sun rise from?',
   'correct_answer': 'East',
   'incorrect_answers': ['West', 'North', 'South']},
  {'type': 'multiple',
   'difficulty': 'hard',
   'category': 'General Knowledge',
   'question': 'The Quadrangularis Reversum is best described as which of the following?',
   'correct_answer': 'A percussion instrument',
   'incorrect_answers': ['A building in Oxford University',
    'A chess move',
    'A geometric theorem']},
  {'type': 'boolean',
   'difficulty': 'easy',
   'category': 'General Knowledge',
   'question': '

See... it is the same url..

In [13]:
response.url

'https://opentdb.com/api.php?amount=10&category=9'

In [14]:
pd.DataFrame(response.json()["results"])

,type,difficulty,category,question,correct_answer,incorrect_answers
0,boolean,easy,General Knowledge,It is automatically considered entrapment in t...,False,[True]
1,multiple,easy,General Knowledge,In which cardinal direction does the Sun rise ...,East,"[West, North, South]"
2,multiple,hard,General Knowledge,The Quadrangularis Reversum is best described ...,A percussion instrument,"[A building in Oxford University, A chess move..."
3,boolean,easy,General Knowledge,The National Animal of Scotland is the Unicorn.,True,[False]
4,multiple,easy,General Knowledge,What was the first ever London Underground lin...,Metropolitan Line,"[Circle Line, Bakerloo Line, Victoria Line]"
5,multiple,hard,General Knowledge,The word &quot;astasia&quot; means which of th...,The inability to stand up,"[The inability to make decisions, The inabilit..."
6,boolean,easy,General Knowledge,"On average, at least 1 person is killed by a d...",True,[False]
7,multiple,medium,General Knowledge,When was WhatsApp founded?,2009,"[2007, 2012, 2010]"
8,multiple,easy,General Knowledge,When someone is inexperienced they are said to...,Green,"[Red, Blue, Yellow]"
9,multiple,easy,General Knowledge,What is the shape of the toy invented by Hunga...,Cube,"[Sphere, Cylinder, Pyramid]"


## Example 2: Yelp API. 

Let's transition now to a more complex, and with interesting data, API. We will work with the Yelp API.

This API: 
-  Requires us to get credentials
-  But does not have a wrapper to query the daya (that I know of). 

See the documentation for the API [here](https://docs.developer.yelp.com/docs/fusion-intro). The API has some interesting endpoints, for example:

- `/businesses/search` - Search for businesses by keyword, category, location, price level, etc.
- `/businesses/{id}` - Get rich business data, such as name, address, phone number, photos, Yelp rating, price levels and hours of operation.
- `/businesses/{business_id_or_alias}/reviews` - Get up to three review excerpts for a business.
- Among many other endpoints

## Authentication with an API

Most often, the provider of an API will require you to authenticate before you can get some data. Authentication usually occures through an access token you can generate directly from the API.  Depending on the type of authentication each API have in place, it can be a simple token (string) or multiple different ids (Client ID, Access Token, Client Token..)

Keep in mind that using a token is better than using a username and password for a few reasons:

- Typically, you'll be accessing an API from a script. If you put your username and password in the script and someone finds it, they can take over your account. 

- Access tokens can have scopes and specific permissions. 

To authorize your access, you need to add the token to your API call. Often, you do this by passing the token through an authorization header. We can use Python's requests library to make a dictionary of headers, and then pass it into our request.


### Acquiring credentials with Yelp Fusion API

Information about acquiring your credentials to make API call are often displayed in the API documentation. 

[Here it is Yelp's information](https://docs.developer.yelp.com/docs/fusion-authentication)

Every API has a bit of a distinct process. In general, APIs require you to create an app to access the API. This is a bit of a weird terminology. The assumption here is that you are creating an app (think about the Botometer at Twitter) that will query the API many times. 

For the YELP API, after you create the app, you will get an `Client ID` and an `API KEY`

### How to save the API keys/token?

API keys are personal information. Keep yours safe, and do not paste into your code.

Don't do this:

`api_key = "my_key"`

Do this:

- create a file with your keys and save as .env
- Add your keys there
- load them in your environment when running the APIs.
- And never upload your .env file in a public server (like github)

I will show you in class what a .env file looks like. 

### Querying the API

We repeat the same steps as before, but adding an authentication step. 

- **Step 0:** Load your API Keys
- **Step 1:** Look at the API documentation and endpoints, and construct a query of interest
- **Step 2:** Use requests.get(querystring) to call the API
- **Step 3:** Examine the response
- **Step 4:** Extract your data and save it. 


### Step 0: Load your API Keys

In [15]:
# load library to get environmental files
import os
from dotenv import load_dotenv


# load keys from  environment variables
load_dotenv() # .env file in cwd

# Print all environment variables
#for key, value in os.environ.items():
#    print(key, "=", value)

yelp_client = os.environ.get("yelp_client_id") 
yelp_key = os.environ.get("yelp_api_key")

# OR JUST HARD CODE YOUR API KEY HERE. NOT A GREAT PRACTICE!!!
#yelp_key = ""

# save your token in the header of the call
header = {'Authorization': f'Bearer {yelp_key}'}

In [16]:
# see here
header["Authorization"]

'Bearer syM5u9r4OFOcdp-ApFx8wD6GEDKaG97kUs9xiO9jQStWvZisnQT3_JENEKYXl6aazVMZAypJPh2g6v4IRHT8viNgXQTObKVWVGQWe_qfiZXVMfs1W047aGAHK9wRZXYx'

### Step 1: Look at the API documentation and endpoints, and construct a query of interest

We will query the `/businesses/search` endpoint. Let's check together the documentation here: https://docs.developer.yelp.com/reference/v3_business_search


We will use two parameters: 

- location: This string indicates the geographic area to be used when searching for businesses
- term: Search term, e.g. "food" or "restaurants".

In [17]:
# endpoint
endpoint = "https://api.yelp.com/v3/businesses/search"

# Add as parameters
params ={"location":" Washington, DC 20057",
        "term":"best noodles restaurant"}

### **Step 2:** Use requests.get(endpoint) to call the API


In [18]:
# Make a get request with header + parameters
response = requests.get(endpoint, 
                        headers=header,
                        params=params)

### **Step 3:** Examine the response

Let's check the response code

In [19]:
# looking for a 200
response.status_code

200

### **Step 4:** Extract your data and save it. 



In [20]:
# What does the response look like?
yelp_json = response.json()

# print
print(yelp_json)

{'businesses': [{'id': 'ZE82dmfQVd3d7V1-xzjK5Q', 'alias': 'yu-noodles-arlington', 'name': 'Yu Noodles', 'image_url': 'https://s3-media0.fl.yelpcdn.com/bphoto/SQfbwgz4Wgl__kIl96FUDQ/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/yu-noodles-arlington?adjust_creative=GJK5eaHUqVE8eGMl0w0Pfg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=GJK5eaHUqVE8eGMl0w0Pfg', 'review_count': 20, 'categories': [{'alias': 'chinese', 'title': 'Chinese'}, {'alias': 'noodles', 'title': 'Noodles'}], 'rating': 4.0, 'coordinates': {'latitude': 38.89511776584536, 'longitude': -77.07472644746304}, 'transactions': ['delivery', 'pickup'], 'location': {'address1': '1515 Wilson Blvd', 'address2': 'Unit 102', 'address3': None, 'city': 'Arlington', 'zip_code': '22209', 'country': 'US', 'state': 'VA', 'display_address': ['1515 Wilson Blvd', 'Unit 102', 'Arlington, VA 22209']}, 'phone': '+17037184928', 'display_phone': '(703) 718-4928', 'distance': 1575.158639233165}, {'id': 'eV_87BqGb

In [21]:
yelp_json.keys()

dict_keys(['businesses', 'total', 'region'])

In [22]:
yelp_json["businesses"]

[{'id': 'ZE82dmfQVd3d7V1-xzjK5Q',
  'alias': 'yu-noodles-arlington',
  'name': 'Yu Noodles',
  'image_url': 'https://s3-media0.fl.yelpcdn.com/bphoto/SQfbwgz4Wgl__kIl96FUDQ/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/yu-noodles-arlington?adjust_creative=GJK5eaHUqVE8eGMl0w0Pfg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=GJK5eaHUqVE8eGMl0w0Pfg',
  'review_count': 20,
  'categories': [{'alias': 'chinese', 'title': 'Chinese'},
   {'alias': 'noodles', 'title': 'Noodles'}],
  'rating': 4.0,
  'coordinates': {'latitude': 38.89511776584536,
   'longitude': -77.07472644746304},
  'transactions': ['delivery', 'pickup'],
  'location': {'address1': '1515 Wilson Blvd',
   'address2': 'Unit 102',
   'address3': None,
   'city': 'Arlington',
   'zip_code': '22209',
   'country': 'US',
   'state': 'VA',
   'display_address': ['1515 Wilson Blvd', 'Unit 102', 'Arlington, VA 22209']},
  'phone': '+17037184928',
  'display_phone': '(703) 718-4928',
  'distance

It returns a long dictionary with the key "businesses" and a list with multiple sub-entries.

**How to deal with this data?**

### Approach 1: Convert all to dataframe and clean it later

In [23]:
# convert to pd
df_yelp = pd.DataFrame(yelp_json["businesses"])

# see
df_yelp

# not looking realy bad. 

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,ZE82dmfQVd3d7V1-xzjK5Q,yu-noodles-arlington,Yu Noodles,https://s3-media0.fl.yelpcdn.com/bphoto/SQfbwg...,False,https://www.yelp.com/biz/yu-noodles-arlington?...,20,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",4.0,"{'latitude': 38.89511776584536, 'longitude': -...","[delivery, pickup]","{'address1': '1515 Wilson Blvd', 'address2': '...",+17037184928,(703) 718-4928,1575.158639,NaN
1,eV_87BqGbpvTqUwjOgQO5g,reren-washington-3,Reren,https://s3-media0.fl.yelpcdn.com/bphoto/PxQkgC...,False,https://www.yelp.com/biz/reren-washington-3?ad...,222,"[{'alias': 'asianfusion', 'title': 'Asian Fusi...",4.0,"{'latitude': 38.90468, 'longitude': -77.06262}","[delivery, pickup]","{'address1': '1073 Wisconsin Ave NW', 'address...",+12028044962,(202) 804-4962,1231.376901,$$
2,hviVXv1CZKWlwbX5JcE2JQ,rimtang-washington,Rimtang,https://s3-media0.fl.yelpcdn.com/bphoto/nHt8Vu...,False,https://www.yelp.com/biz/rimtang-washington?ad...,46,"[{'alias': 'thai', 'title': 'Thai'}]",4.4,"{'latitude': 38.9047, 'longitude': -77.06586}","[delivery, pickup]","{'address1': '1039 33rd St NW', 'address2': ''...",,,980.131987,NaN
3,RiwIUBITUfhn6etk6qVbnQ,uyghur-eats-washington,Uyghur Eats,https://s3-media0.fl.yelpcdn.com/bphoto/GbQu1A...,False,https://www.yelp.com/biz/uyghur-eats-washingto...,204,"[{'alias': 'noodles', 'title': 'Noodles'}, {'a...",4.3,"{'latitude': 38.92138, 'longitude': -77.07251}","[delivery, restaurant_reservation, pickup]","{'address1': '2412 Wisconsin Ave NW', 'address...",+12023333600,(202) 333-3600,1371.802931,$$
4,Ek_-kvajIvVJbi3ll4pMww,pho-75-arlington,Pho 75,https://s3-media0.fl.yelpcdn.com/bphoto/g-xbss...,False,https://www.yelp.com/biz/pho-75-arlington?adju...,2157,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",4.1,"{'latitude': 38.8941969403826, 'longitude': -7...",[delivery],"{'address1': '1721 Wilson Blvd', 'address2': N...",+17035257355,(703) 525-7355,1699.743009,$$
5,QanUICteMAzlK7jVADa1JA,oki-shoten-washington,OKI Shoten,https://s3-media0.fl.yelpcdn.com/bphoto/w4kLjE...,False,https://www.yelp.com/biz/oki-shoten-washington...,309,"[{'alias': 'ramen', 'title': 'Ramen'}]",4.0,"{'latitude': 38.911279798268254, 'longitude': ...","[pickup, delivery]","{'address1': '1614 Wisconsin Ave NW', 'address...",+12029448660,(202) 944-8660,887.458472,$$
6,HYu17SsplcpRLDjseFfZ_g,bangbop-washington,Bangbop,https://s3-media0.fl.yelpcdn.com/bphoto/FRTOxw...,False,https://www.yelp.com/biz/bangbop-washington?ad...,98,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.6,"{'latitude': 38.90689510951072, 'longitude': -...",[pickup],"{'address1': '4418 MacArthur Blvd NW', 'addres...",+12023500846,(202) 350-0846,627.043245,$$
7,Lx9N4_5bWxfLTU3f-_8YBw,dumplings-and-beyond-washington,Dumplings & Beyond,https://s3-media0.fl.yelpcdn.com/bphoto/zWuV4m...,False,https://www.yelp.com/biz/dumplings-and-beyond-...,627,"[{'alias': 'chinese', 'title': 'Chinese'}]",4.1,"{'latitude': 38.9211850288095, 'longitude': -7...","[delivery, pickup]","{'address1': '2400 Wisconsin Ave NW', 'address...",+12023383815,(202) 338-3815,1353.517869,$$
8,IpXiBHLfL1hRyPEIoljA7A,ramen-by-uzu-washington-5,Ramen By Uzu,https://s3-media0.fl.yelpcdn.com/bphoto/qChXwu...,False,https://www.yelp.com/biz/ramen-by-uzu-washingt...,13,"[{'alias': 'ramen', 'title': 'Ramen'}, {'alias...",5.0,"{'latitude': 38.9038, 'longitude': -77.06351}","[delivery, pickup]","{'address1': '3210 Grace St NW', 'address2': N...",,,1208.638684,NaN
9,XKOVFGUCK1e0vZBML4ddxw,donsak-thai-restaurant-washington,Donsak Thai Restaurant,https://s3-media0.fl.yelpcdn.com/bphoto/btbTD5...,False,https://www.yelp.com/biz/donsak-thai-restauran...,132,"[{'alias': 'thai', 'title': 'Thai'}]",4.3,"{'latitude': 38.92398, 'longitude': -77.05212}","[delivery, pickup]","{'address1': '2608 Connecticut Ave NW', 'addre...",+12025078207,(202) 507-8207,2610.575513,NaN


### Approach 2: write a function to collect the information you need

Assume you are interested in the id, name, url, lat and long, and rating

In [24]:
# function to clean and extract information from yelp
def clean_yelp(yelp_json):
    '''
    function to extract columns of interest from yelp json
    '''
    # create a temporary dictionary to store the information
    temp_yelp = {}
    
    # collect information
    temp_yelp["id"]= yelp_json["id"]
    temp_yelp["name"]= yelp_json["name"]
    temp_yelp["url"]= yelp_json["url"]
    temp_yelp["latitude"] = yelp_json["coordinates"]["latitude"]
    temp_yelp["longitude"] = yelp_json["coordinates"]["longitude"]
    temp_yelp["rating"]= yelp_json["rating"]
    
    # return
    
    return(temp_yelp)
    

In [26]:
# apply to the dictionary
results_yelp = [clean_yelp(entry) for entry in yelp_json["businesses"]]

# Convert results to dataframe
yelp_df = pd.DataFrame(results_yelp)   
yelp_df

,id,name,url,latitude,longitude,rating
0,ZE82dmfQVd3d7V1-xzjK5Q,Yu Noodles,https://www.yelp.com/biz/yu-noodles-arlington?...,38.895118,-77.074726,4.0
1,eV_87BqGbpvTqUwjOgQO5g,Reren,https://www.yelp.com/biz/reren-washington-3?ad...,38.904680,-77.062620,4.0
2,hviVXv1CZKWlwbX5JcE2JQ,Rimtang,https://www.yelp.com/biz/rimtang-washington?ad...,38.904700,-77.065860,4.4
3,RiwIUBITUfhn6etk6qVbnQ,Uyghur Eats,https://www.yelp.com/biz/uyghur-eats-washingto...,38.921380,-77.072510,4.3
4,Ek_-kvajIvVJbi3ll4pMww,Pho 75,https://www.yelp.com/biz/pho-75-arlington?adju...,38.894197,-77.078854,4.1
5,QanUICteMAzlK7jVADa1JA,OKI Shoten,https://www.yelp.com/biz/oki-shoten-washington...,38.911280,-77.065646,4.0
6,HYu17SsplcpRLDjseFfZ_g,Bangbop,https://www.yelp.com/biz/bangbop-washington?ad...,38.906895,-77.082146,4.6
7,Lx9N4_5bWxfLTU3f-_8YBw,Dumplings & Beyond,https://www.yelp.com/biz/dumplings-and-beyond-...,38.921185,-77.072382,4.1
8,IpXiBHLfL1hRyPEIoljA7A,Ramen By Uzu,https://www.yelp.com/biz/ramen-by-uzu-washingt...,38.903800,-77.063510,5.0
9,XKOVFGUCK1e0vZBML4ddxw,Donsak Thai Restaurant,https://www.yelp.com/biz/donsak-thai-restauran...,38.923980,-77.052120,4.3


#### Save the json

Remember to always save your response from the API call. You don't want be querying the API all the time to grab the same data. 

In [118]:
import json

with open("yelp_results.json", 'w') as f:
    # write the dictionary to a string
    json.dump(response.json(), f, indent=4)

## Practice

Make a successful query using your favorite type of food to the Yelp API. Pretty much I only want you to repeat what we did before, but changing the search term a bit 

In [ ]:
# code here

## Example 3 : YouTube API

Now let's move to our last example. 

We will be working with the YouTube API. This is a complex API, but lucky for us some other programmers already created a Python wrapper to access the API. We will use the [youtube-data-api](https://youtube-data-api.readthedocs.io/en/latest/youtube_api.html) library which contains a set of functions to facilitate the access to the API. 

### What kind of data can you get from the Youtube API?

Youtube has a very extensive api. There are a lot of data you can get access to. See a compreensive list [here](https://developers.google.com/youtube/v3/docs/)

What is included in the package:

- video metadata
- channel metadata
- playlist metadata
- subscription metadata
- featured channel metadata
- comment metadata
- search results

### How to Install

The software is on PyPI, so you can download it via `pip`
   

In [60]:
#!pip install youtube-data-api

### How to get an API key

#### A quick guide: [https://developers.google.com/youtube/v3/getting-started](https://developers.google.com/youtube/v3/getting-started)

1. You need a Google Account to access the Google API Console, request an API key, and register your application. You can use your GMail account for this if you have one.

2. Create a project in the <a href="https://console.developers.google.com/apis/">Google Developers Console</a> and <a href="https://developers.google.com/youtube/registering_an_application">obtain authorization credentials</a> so your application can submit API requests.

3. After creating your project, make sure the YouTube Data API is one of the services that your application is registered to use.

    a. Go to the <a href="https://console.developers.google.com/apis/">API Console</a> and select the project that you just registered.

    b. Visit the <a href="https://console.developers.google.com/apis/enabled">Enabled APIs page</a>. In the list of APIs, make sure the status is ON for the YouTube Data API v3. You do not need to enable OAuth 2.0 since there are no methods in the package that require it.
        

In [27]:
# call some libraries
import os
import datetime
import pandas as pd

In [28]:
#Import YouTubeDataAPI
from youtube_api import YouTubeDataAPI
from youtube_api.youtube_api_utils import *
from dotenv import load_dotenv

In [29]:
# load keys from  environmental var
load_dotenv() # .env file in cwd
api_key = os.environ.get("YT_KEY")

In [30]:
# create a client 
# this is what we call: instantiate the class
yt = YouTubeDataAPI(api_key)
print(yt)

#### Starting with a channel name and getting some basic metadata

Let's start with the `LastWeekTonight` channel

[https://www.youtube.com/user/LastWeekTonight](https://www.youtube.com/user/LastWeekTonight)

First we need to get the channel id

In [31]:
channel_id = yt.get_channel_id_from_user('LastWeekTonight')
print(channel_id)

UC3XTzVzaHQEd30rQbuvCtTQ


#### Channel metadata

In [32]:
# collect metadata
yt.get_channel_metadata(channel_id)

{'channel_id': 'UC3XTzVzaHQEd30rQbuvCtTQ',
 'title': 'LastWeekTonight',
 'account_creation_date': 1395178899.0,
 'keywords': None,
 'description': 'Breaking news on a weekly basis. Sundays at 11PM - only on HBO.\nSubscribe to the Last Week Tonight channel for the latest videos from John Oliver and the LWT team.',
 'view_count': '4514448979',
 'video_count': '899',
 'subscription_count': '10000000',
 'playlist_id_likes': '',
 'playlist_id_uploads': 'UU3XTzVzaHQEd30rQbuvCtTQ',
 'topic_ids': 'https://en.wikipedia.org/wiki/Politics|https://en.wikipedia.org/wiki/Television_program|https://en.wikipedia.org/wiki/Society|https://en.wikipedia.org/wiki/Entertainment',
 'country': None,
 'collection_date': datetime.datetime(2025, 10, 27, 14, 32, 2, 66639)}

#### Subscriptions of the channel. 

In [33]:
pd.DataFrame(yt.get_subscriptions(channel_id))

,subscription_title,subscription_channel_id,subscription_kind,subscription_publish_date,collection_date
0,trueblood,UCPnlBOg4_NU9wdhRN-vzECQ,youtube#channel,1.395357e+09,2025-10-27 14:32:05.254688
1,GameofThrones,UCQzdMyuz0Lf4zo4uGcEujFw,youtube#channel,1.395357e+09,2025-10-27 14:32:05.254772
2,HBO,UCVTQuK2CaWaTgSsoNkn5AiQ,youtube#channel,1.395357e+09,2025-10-27 14:32:05.254944
3,HBOBoxing,UCWPQB43yGKEum3eW0P9N_nQ,youtube#channel,1.395357e+09,2025-10-27 14:32:05.254997
4,Cinemax,UCYbinjMxWwjRpp4WqgDqEDA,youtube#channel,1.424812e+09,2025-10-27 14:32:05.255022
5,HBODocs,UCbKo3HsaBOPhdRpgzqtRnqA,youtube#channel,1.395357e+09,2025-10-27 14:32:05.255043
6,HBOLatino,UCeKum6mhlVAjUFIW15mVBPg,youtube#channel,1.395357e+09,2025-10-27 14:32:05.255065
7,OfficialAmySedaris,UCicerXLHzJaKYHm1IwvTn8A,youtube#channel,1.461561e+09,2025-10-27 14:32:05.255110
8,Real Time with Bill Maher,UCy6kyFxaMqGtpE3pQTflK8A,youtube#channel,1.418342e+09,2025-10-27 14:32:05.255129


#### List of videos of the channel
You first need to convert the `channel_id` into a playlist id to get all the videos ever posted by a channel using a function from the `youtube_api_utils` in the package. Then you can get the video ids, and collect metadata, comments, among many others. 

In [34]:
from youtube_api.youtube_api_utils import *
playlist_id = get_upload_playlist_id(channel_id)
print(playlist_id)

UU3XTzVzaHQEd30rQbuvCtTQ


In [35]:
## Get video ids
videos = yt.get_videos_from_playlist_id(playlist_id)
df = pd.DataFrame(videos)
print(df)

        video_id                channel_id  publish_date  \
0    Mwc21oNdnaA  UC3XTzVzaHQEd30rQbuvCtTQ  1.761601e+09   
1    Ejoi9yfLVCc  UC3XTzVzaHQEd30rQbuvCtTQ  1.761561e+09   
2    73iQpsIE0i8  UC3XTzVzaHQEd30rQbuvCtTQ  1.761548e+09   
3    m9ExweRMFAA  UC3XTzVzaHQEd30rQbuvCtTQ  1.761000e+09   
4    s9FsxWK0f1A  UC3XTzVzaHQEd30rQbuvCtTQ  1.760944e+09   
..           ...                       ...           ...   
893  Dh9munYYoqQ  UC3XTzVzaHQEd30rQbuvCtTQ  1.398670e+09   
894  k8lJ85pfb_E  UC3XTzVzaHQEd30rQbuvCtTQ  1.398669e+09   
895  WHCQndalv94  UC3XTzVzaHQEd30rQbuvCtTQ  1.398663e+09   
896  8q7esuODnQI  UC3XTzVzaHQEd30rQbuvCtTQ  1.395379e+09   
897  gdQCtWlhx90  UC3XTzVzaHQEd30rQbuvCtTQ  1.395379e+09   

               collection_date  
0   2025-10-27 14:32:07.951453  
1   2025-10-27 14:32:07.951516  
2   2025-10-27 14:32:07.951584  
3   2025-10-27 14:32:07.951634  
4   2025-10-27 14:32:07.951679  
..                         ...  
893 2025-10-27 14:32:10.075527  
894 2025-10-27 

#### Collect video metadata

In [37]:
# id for videos as a list
df.video_id.tolist()

['Mwc21oNdnaA',
 'Ejoi9yfLVCc',
 '73iQpsIE0i8',
 'm9ExweRMFAA',
 's9FsxWK0f1A',
 '-xIQxzlXN-0',
 'cicEeYtFd1M',
 'a277Pg23Dao',
 '-6r-lAxhtv4',
 'bl6Ww92bb0o',
 'gieTx_P6INQ',
 '9iZK_DurYOo',
 'xQwGv4UYvbk',
 '88YixeXbRMo',
 'eHJwoYjTyyE',
 'SCv0hlq5iQY',
 'JOoHELC8w8M',
 'dB1-lg-xZWc',
 'NtHZ7IR88dg',
 'S9EVYaSa1Ws',
 'Wg8OcJopuBE',
 'z016SEN7HzE',
 'wn46weQY0DY',
 '0_Bwix9IjOE',
 'G97ew5P68vI',
 '8ek4E7vkiQw',
 'KU5eY7wBkNI',
 'IhmjgncyBiI',
 'k315NvOdHvc',
 'hSJtzwy6E_M',
 'ohPToBog_-g',
 'gZlw0gKXQAI',
 'I9yskcTgalA',
 'osh1v6FqUhc',
 'xk94il8L820',
 '4T-CHSNHx6U',
 'dijMKwZMU2Q',
 'eP5vqvGLC1I',
 'W3V1oswZM-o',
 '7wdGf-48OKo',
 'MEtfL3fBaRU',
 'kSZ9DNj5_Ts',
 'lcbnhXKvi-c',
 '3lzfH86avIc',
 'E02gokpa7Ug',
 'o6g62JCgWxc',
 'VPtAWwRBHUc',
 'VJAIChXYQVk',
 'ZSTM7-rq4eE',
 'DfTBhrkae74',
 '2psRpQvFSno',
 'lrZlBPJYH-Y',
 'yQ_xxbVtcYo',
 'eYjuA0Yc-7Y',
 '_R3uMbUyyaI',
 'xNo8Ve-Ej6U',
 'TMk28nV9nkI',
 'otn03GFKpgE',
 '7GwfM53wqak',
 'qjafPzoy4eI',
 'ZTgI0mTwUCs',
 'XBAtPdMej98',
 'R_VsQd

In [38]:
#grab metadata
video_meta = yt.get_video_metadata(df.video_id.tolist()[:5])

In [39]:
#visualize
pd.DataFrame(video_meta)

,video_id,channel_title,channel_id,video_publish_date,video_title,video_description,video_category,video_view_count,video_comment_count,video_like_count,video_dislike_count,video_thumbnail,video_tags,collection_date
0,Mwc21oNdnaA,LastWeekTonight,UC3XTzVzaHQEd30rQbuvCtTQ,1.761601e+09,Trump & Military Strikes #lastweektonight,Congress should be reviewing Trump’s plans for...,24,11255,64,1260,None,https://i.ytimg.com/vi/Mwc21oNdnaA/hqdefault.jpg,,2025-10-27 14:32:13.922271
1,Ejoi9yfLVCc,LastWeekTonight,UC3XTzVzaHQEd30rQbuvCtTQ,1.761561e+09,Medicare Advantage: Last Week Tonight with Joh...,John Oliver details what Medicare Advantage is...,24,996788,4113,45611,None,https://i.ytimg.com/vi/Ejoi9yfLVCc/hqdefault.jpg,,2025-10-27 14:32:13.922341
2,73iQpsIE0i8,LastWeekTonight,UC3XTzVzaHQEd30rQbuvCtTQ,1.761548e+09,S12 E27: Trump’s Week & Medicare Advantage: 10...,John Oliver discusses the many bold moves Dona...,24,203578,736,7799,None,https://i.ytimg.com/vi/73iQpsIE0i8/hqdefault.jpg,,2025-10-27 14:32:13.922422
3,m9ExweRMFAA,LastWeekTonight,UC3XTzVzaHQEd30rQbuvCtTQ,1.761000e+09,Air Bud Pt. II #lastweektonight,That should sum it up! Please watch our full s...,24,404980,126,12733,None,https://i.ytimg.com/vi/m9ExweRMFAA/hqdefault.jpg,,2025-10-27 14:32:13.922467
4,s9FsxWK0f1A,LastWeekTonight,UC3XTzVzaHQEd30rQbuvCtTQ,1.760944e+09,Air Bud Pt. II: Last Week Tonight with John Ol...,John Oliver discusses the Air Bud franchise. A...,24,2600908,4936,86745,None,https://i.ytimg.com/vi/s9FsxWK0f1A/hqdefault.jpg,,2025-10-27 14:32:13.922509


In [40]:
## Collect Comments
ids = df.video_id.tolist()[:5]

In [41]:
ids

['Mwc21oNdnaA', 'Ejoi9yfLVCc', '73iQpsIE0i8', 'm9ExweRMFAA', 's9FsxWK0f1A']

In [42]:
# loop
list_comments = []

for video_id in ids:
    comments = yt.get_video_comments(video_id, max_results=10)
    list_comments.append(pd.DataFrame(comments))

# concat
df = pd.concat(list_comments)
df.head()

,video_id,commenter_channel_url,commenter_channel_id,commenter_channel_display_name,comment_id,comment_like_count,comment_publish_date,text,commenter_rating,comment_parent_id,collection_date,reply_count
0,Mwc21oNdnaA,http://www.youtube.com/@galimirnund6543,UCOk3nlK804t82lxHW2Imkgw,@galimirnund6543,UgwOfRBX2qIe7MhgYa14AaABAg,0,1.761604e+09,Murderer.... anyone else would be tried and go...,none,None,2025-10-27 14:32:18.662247,0
1,Mwc21oNdnaA,http://www.youtube.com/@CantFightRobots,UCZrgUVTENMWcsPFyjADC2OQ,@CantFightRobots,UgwxhMx-dwzFTi-u51p4AaABAg,0,1.761604e+09,And they say women are emotional. Trump is the...,none,None,2025-10-27 14:32:18.662294,0
2,Mwc21oNdnaA,http://www.youtube.com/@perfectpainter9140,UC7zlVykVcW-Rq9ZxWyXUs2g,@perfectpainter9140,UgxMWr18QQbeXoHFKul4AaABAg,0,1.761604e+09,This aren’t drug boats,none,None,2025-10-27 14:32:18.662327,0
3,Mwc21oNdnaA,http://www.youtube.com/@matthiaseriklutge8652,UC-RTTt5ihwDR9kihSoS6Hdg,@matthiaseriklutge8652,UgwsO_0T6jNLKUYkrup4AaABAg,2,1.761604e+09,30 billion to Argentina? where are the people ...,none,None,2025-10-27 14:32:18.662357,0
4,Mwc21oNdnaA,http://www.youtube.com/@patriciagarrison3182,UCPVAAl44CTRAccUklGdzQKQ,@patriciagarrison3182,Ugzsh5jwVRGNW41BCu14AaABAg,0,1.761604e+09,Please…please…please…someone save us from this...,none,None,2025-10-27 14:32:18.662385,0


#### Search

The youtube API also allows you to search for most popular videos using queries. This is very cool!


In [43]:
df = pd.DataFrame(yt.search(q='they are eating the dogs', max_results=10))
df.keys()
df

,video_id,channel_title,channel_id,video_publish_date,video_title,video_description,video_category,video_thumbnail,collection_date
0,lHycpIhnFcU,WSJ News,UCMliswJ7oukCeW35GSayhRA,1.726052e+09,‘They&#39;re Eating the Dogs:&#39; Trump Makes...,Former President Donald Trump falsely claimed ...,None,https://i.ytimg.com/vi/lHycpIhnFcU/hqdefault.jpg,2025-10-27 14:32:24.029904
1,5llMaZ80ErY,Atlanta News First,UCWElIIlFx2JiyI9UVrbrEcw,1.726037e+09,"Trump: They&#39;re eating the dogs, the cats",Former President Donald Trump claims immigrant...,None,https://i.ytimg.com/vi/5llMaZ80ErY/hqdefault.jpg,2025-10-27 14:32:24.029945
2,3BrCvZmSnKA,The Kiffness,UCFy846QdKs3LbLgBpSqPcdg,1.726247e+09,The Kiffness - Eating the Cats ft. Donald Trum...,"Stream / Buy ""Eating the Cats"" here: https://o...",None,https://i.ytimg.com/vi/3BrCvZmSnKA/hqdefault.jpg,2025-10-27 14:32:24.029973
3,V5CR7HkByC4,CNBC Television,UCrp_UI8XtuYfpiqluWLD7Lw,1.726040e+09,Former President Trump claims immigrants are &...,Vice President Harris and Former President Tru...,None,https://i.ytimg.com/vi/V5CR7HkByC4/hqdefault.jpg,2025-10-27 14:32:24.030001
4,9SeMHr-sYuM,Leonard Eisen,UCe-sn976Wq5G3mB7V3mZmhg,1.726111e+09,Trump - &quot;They&#39;re Eating the Dogs&quot...,Insert joke about RFK Jr eating that dog carca...,None,https://i.ytimg.com/vi/9SeMHr-sYuM/hqdefault.jpg,2025-10-27 14:32:24.030026
5,dX8vJl9Zqno,AriaisDeluX,UC1tPMsySD3aL3fSMYmM57vA,1.760491e+09,Dogs vs cats when they want some of your food....,,None,https://i.ytimg.com/vi/dX8vJl9Zqno/hqdefault.jpg,2025-10-27 14:32:24.030123
6,IStRiNCfbkw,The Sean Ward Show,UCEvLUnQiH-i1XUNikh0Zm4g,1.726512e+09,They&#39;re Eating the Dogs! They&#39;re Eatin...,They're Eating the Dogs! They're Eating the Ca...,None,https://i.ytimg.com/vi/IStRiNCfbkw/hqdefault.jpg,2025-10-27 14:32:24.030177
7,vpbBGHiWF8M,Eevee Bork the Shiba,UCMAAOw1cq5z2xX5QZaIPQaw,1.726259e+09,"They’re EATING the DOGS, they’re EATING the CA...","They're eating the dogs, they're eating the ca...",None,https://i.ytimg.com/vi/vpbBGHiWF8M/hqdefault.jpg,2025-10-27 14:32:24.030291
8,Uq7FcWLDEtk,WatchMojo.com,UCaWd5_7JhbQBe4dknZhsHJg,1.726177e+09,They&#39;re Eating The Dogs In Springfield!,The second US debate gave us lots of unbelieva...,None,https://i.ytimg.com/vi/Uq7FcWLDEtk/hqdefault.jpg,2025-10-27 14:32:24.030334
9,yx4e0hFmzzg,Eyewitness News ABC7NY,UCrlIS7z20CnVaCrMvdkig_g,1.726041e+09,Trump pushes false claim that Haitian migrants...,Check out more Eyewitness News - http://abc7ny...,None,https://i.ytimg.com/vi/yx4e0hFmzzg/hqdefault.jpg,2025-10-27 14:32:24.030364


Some cool research using the Youtube API: 
    
- [Lei et al, Estimating the Ideology of Political YouTube Videos](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4088828)

- [Brown et al, Echo Chambers, Rabbit Holes, and Algorithmic Bias: How YouTube Recommends Content to Real Users](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4114905)
    

In [44]:
!jupyter nbconvert _week-08_apis.ipynb --to html --template classic


[NbConvertApp] Converting notebook _week-08_apis.ipynb to html
[NbConvertApp] Writing 461383 bytes to _week-08_apis.html
